In [1]:
import json
from io import StringIO
from datetime import datetime
import os

In [2]:
import panel as pn
pn.extension()

In [3]:
'json.__version__', json.__version__, 'panel', pn.__version__

('json.__version__', '2.0.9', 'panel', '0.14.0')

In [8]:
output_w = pn.widgets.TextInput(value='') 



class samples:
    
    def __init__(self):
    
        self.sample_dict = {}
        self.sample_table = []
        self.sample_widgets = {}
        
        self.titles = pn.Row( pn.widgets.StaticText(value='Block Position', 
                                                    min_width=80, max_width=80),
                           pn.widgets.StaticText( value='First Name',  
                                                 min_width=70, max_width=70),
                           pn.widgets.StaticText(value='Last Name',
                                                 min_width=70, max_width=180),
                           pn.widgets.StaticText(value='Email', 
                                                 min_width=180, max_width=180),
                            pn.widgets.StaticText(value='Sample ID', 
                                                  min_width=70, max_width=70),
                            pn.widgets.StaticText(value='Solvent', 
                                                  min_width=70, max_width=70),
                            pn.widgets.StaticText(value='Experiments', 
                                                  min_width=70, max_width=70))
        
        
        
        self.widget = pn.Column(self.titles)
        

    
    def add_row(self, row_data, makebckup=True):
        output_w.value = 'add_row'
        self.sample_dict[int(row_data['blockpos'])] = row_data
        self.sample_widgets[int(row_data['blockpos'])] = pn.Row( pn.widgets.StaticText(value=row_data['blockpos'], 
                                                                min_width=80, max_width=80),
                                           pn.widgets.StaticText( value=row_data['firstname'], 
                                                                 min_width=70,  max_width=70),
                                           pn.widgets.StaticText(value=row_data['lastname'], 
                                                                 min_width=70, max_width=70),
                                           pn.widgets.StaticText(value=row_data['email'], min_width=180, max_width=180),
                                           pn.widgets.StaticText(value=row_data['sampleid'], 
                                                                 min_width=70, max_width=70),
                                           pn.widgets.StaticText(value=row_data['solvent'], 
                                                                 min_width=70, max_width=70),
                                           pn.widgets.StaticText(value=row_data['experiment'], 
                                                                 min_width=70, max_width=70),
                                           pn.widgets.Button(name='Delete ' + row_data['blockpos'], max_width=60))
        
        sss.sample_widgets[int(row_data['blockpos'])][-1].on_click(self.deleterow_e)
    
        self.widget.clear()
        self.widget.append(self.titles)
        
        kkk = list(self.sample_widgets.keys())
        kkk.sort()
        
        for k in kkk:

            self.widget.append(self.sample_widgets[k])
            
            
        if makebckup:
            
            now = datetime.now()
            timestamp = 'bckup_' + now.strftime("%Y%m%d%H%M%S") + '.json'

            output_w.value = timestamp

            with open(timestamp, 'w') as fp:
                json.dump(self.sample_dict, fp, indent=2)
            
        

    def deleterow_e(self, event):
        _, row_num = event.obj.name.split()
        output_w.value = row_num
    
        self.deleterow(int(row_num))
    
    
    def deleterow(self, row_num):
        output_w.value = 'deleterow'

        self.sample_dict.pop(row_num)
        self.sample_widgets.pop(row_num)
        self.widget.clear()
        self.widget.append(self.titles)
        
        kkk = list(self.sample_widgets.keys())
        kkk.sort()
        
        for k in kkk:
            self.widget.append(self.sample_widgets[k])
            
            
        now = datetime.now()
        timestamp = 'bckup_' + now.strftime("%Y%m%d%H%M%S") + '.json'
            
        with open(timestamp, 'w') as fp:
            json.dump(self.sample_dict, fp, indent=2)
            
            
    def create_nmr_output_file(self):

        nmrsamples = self.sample_dict

        rw = []
        secs = 60
        now = datetime.now()
        timestamp = now.strftime("%d%H%M")
        # // Start creating iconNMR automation file by putting each line in an array

        rw.append('USER walkup')
        rw.append('#')

        # sort keys in numerical order

        kkk = list(nmrsamples.keys())
        kkk.sort()



        samplenumber = 1
        for k in kkk:

            rw.append('HOLDER ' + str(samplenumber))
            samplenumber += 1

            rw.append('NAME ' + timestamp + str(secs))
            secs += 1
            rw.append('EXPNO 10')
            rw.append('SOLVENT ' + nmrsamples[k]['solvent'])

            expts = nmrsamples[k]['experiment']

            for e in expts:
                if 'H1' in e:
                    if len(expts) == 2:
                        rw.append('EXPERIMENT GRUBBS.d')
                    else:
                        rw.append('EXPERIMENT PROTON.d')
                else:
                    rw.append('EXPERIMENT P31CPD.d')

                rw.append('TITLE walkup:' + 
                          nmrsamples[k]['firstname'] + 
                          ' ' + 
                          nmrsamples[k]['lastname'] + 
                          ':' +
                          nmrsamples[k]['sampleid'] + 
                          ', ' +
                          nmrsamples[k]['email'] + 
                          ', Block Pos: ' + 
                          nmrsamples[k]['blockpos'])
                
            rw.append('#')
        rw.append('END')

        #     print(expts)

        nmroutput_str = "\n".join(rw)

        return nmroutput_str

In [9]:
sss = samples()

nmrexpt_w = pn.widgets.MultiChoice(name='NMR Experiment', 
                                   value=['H1'],
                                    options=['H1', 'P31'],
                                   width=150
                                  )


blockPosition_w = pn.widgets.IntInput(name='Block Position:', start=1, end=60, 
                                      min_width=80, max_width=80)
studentFirstname_w = pn.widgets.TextInput(name='First Name:',
                                          min_width=80, max_width=80)
studentLastname_w = pn.widgets.TextInput(name='Last Name:',
                                         min_width=80, max_width=80)
studentEmail_w = pn.widgets.TextInput(name='Email:',
                                      min_width=80, max_width=80)
sampleID_w = pn.widgets.TextInput(name='Sample ID:',
                                  min_width = 80, max_width=80)
addrow_w = pn.widgets.Button(name='Add', 
                             width=100, 
                             height=100)

deleterow_w = pn.widgets.Button(name='Delete', max_width=60)

def create_bruker_auto_file():
    
    sio = StringIO(sss.create_nmr_output_file())
    
    

#     json.dump(sss.sample_dict, sio, indent=2)
    sio.seek(0)
    return sio

submit_w = pn.widgets.FileDownload(label='Submit', callback=create_bruker_auto_file, filename='test.txt')

    

# @pn.depends(years, mpg)
# def filtered_file(yr, mpg):
#     df = filtered_mpg(yr, mpg)
#     sio = StringIO()
#     df.to_csv(sio)
#     sio.seek(0)
#     return sio

# fd = pn.widgets.FileDownload(
#     callback=filtered_file, filename='filtered_autompg.csv'
# )


solvent_w = pn.widgets.Select(name='Solvent:', 
                              options=['CDCl3', 'CDCl3', 'CDCl3', 'CDCl3'], 
                              width=100,)


    
    
def addrow_e(event):

    
    row_data = {'blockpos': str(blockPosition_w.value),
                'firstname': studentFirstname_w.value,
                'lastname': studentLastname_w.value,
                'email': studentEmail_w.value,
                'sampleid': sampleID_w.value,
                'solvent': solvent_w.value,
                'experiment': nmrexpt_w.value}
    
    sss.add_row(row_data)
    
    blockPosition_w.value += 1
    
    output_w.value = ",".join([str(blockPosition_w.value),
                               studentFirstname_w.value,
                              studentLastname_w.value,
                              studentEmail_w.value,
                              sampleID_w.value,
                              solvent_w.value,
                               ]+nmrexpt_w.value)
addrow_w.on_click(addrow_e)



row = pn.Row(
             blockPosition_w,
             studentFirstname_w,
             studentLastname_w,
             studentEmail_w,
             sampleID_w,
             solvent_w,
             nmrexpt_w, 
             addrow_w,
             background='WhiteSmoke')

# df = pd.dataframe()

# df_w = pn.widgets.DataFrame(df, name='DataFrame')


page2 = pn.Column(row, output_w, sss.widget, submit_w)

In [12]:
def get_backupfile_list():
    
    bckup_files = [f for f in os.listdir() if 'json' in f]
    print(bckup_files)
    bckup_files.sort(reverse=True)
#     return bckup_files[-10:]
    return bckup_files[:10]
    
    
def restoreBackup_e(event):
    
    bckup_filename = backupFiles_w.value
    print(bckup_filename)
    if os.path.exists(bckup_filename):
        print(bckup_filename)
        with open(bckup_filename, 'r') as fp:
            sample_dict = json.load(fp)
            
    print(sample_dict)
    
    sss.sample_dict.clear()
    sss.sample_widgets.clear()
    blckpos = 1
    for k,v in sample_dict.items():
        print('sample')
        print(k,v)
        blckpos = int(v['blockpos'])
        sss.add_row(v, makebckup=False)
        
    blckpos += 1
    blockPosition_w.value = blckpos
    
    
def reloadBackup_e(event):
    
    backupFiles_w.options = get_backupfile_list()
    
    

backupFiles_w = pn.widgets.Select(name='Backup Files', options=get_backupfile_list())
restoreBackup_w = pn.widgets.Button(name='Restore')
reloadBackup_w = pn.widgets.Button(name='Reload List')

restoreBackup_w.on_click(restoreBackup_e)
reloadBackup_w.on_click(reloadBackup_e)



page1 = pn.Column(backupFiles_w, restoreBackup_w, reloadBackup_w)


['bckup_20220907113513.json', 'bckup_20220907113518.json', 'bckup_20220907113523.json', 'bckup_20220907113528.json', 'bckup_20220907113533.json', 'bckup_20220907113539.json', 'bckup_20220907113544.json', 'bckup_20220907113550.json', 'bckup_20220907115538.json', 'bckup_20220907115551.json', 'bckup_20230209161440.json', 'bckup_20230209161447.json', 'bckup_20230209161449.json', 'bckup_20230209161450.json', 'bckup_20230209161454.json', 'bckup_20230209161456.json', 'bckup_20230209161458.json']


In [13]:
tabs = pn.Tabs(('Start', page1), ('NMR samples', page2))
tabs.servable()

['bckup_20220907113513.json', 'bckup_20220907113518.json', 'bckup_20220907113523.json', 'bckup_20220907113528.json', 'bckup_20220907113533.json', 'bckup_20220907113539.json', 'bckup_20220907113544.json', 'bckup_20220907113550.json', 'bckup_20220907115538.json', 'bckup_20220907115551.json', 'bckup_20230209161440.json', 'bckup_20230209161447.json', 'bckup_20230209161449.json', 'bckup_20230209161450.json', 'bckup_20230209161454.json', 'bckup_20230209161456.json', 'bckup_20230209161458.json']

Tabs
    [0] Column
        [0] Select(name='Backup Files', options=['bckup_20230209161458.jso...], value='bckup_20230209161458.json...)
        [1] Button(name='Restore')
        [2] Button(name='Reload List')
    [1] Column
        [0] Row(background='WhiteSmoke')
            [0] IntInput(end=60, max_width=80, min_width=80, name='Block Position:', start=1, value=3)
            [1] TextInput(max_width=80, min_width=80, name='First Name:')
            [2] TextInput(max_width=80, min_width=80, name='Last Name:')
            [3] TextInput(max_width=80, min_width=80, name='Email:')
            [4] TextInput(max_width=80, min_width=80, name='Sample ID:')
            [5] Select(name='Solvent:', options=['CDCl3', 'CDCl3', ...], value='CDCl3', width=100)
            [6] MultiChoice(name='NMR Experiment', options=['H1', 'P31'], value=['H1'], width=150)
            [7] Button(height=100, name='Add', sizing_mode='fixed', width=100)
        [1] TextInput(value='add_row')
        [2] Column
            [0] Row
                [0] StaticText(max_width=80, min_width=80, value='Block Position')
                [1] StaticText(max_width=70, min_width=70, value='First Name')
                [2] StaticText(max_width=180, min_width=70, value='Last Name')
                [3] StaticText(max_width=180, min_width=180, value='Email')
                [4] StaticText(max_width=70, min_width=70, value='Sample ID')
                [5] StaticText(max_width=70, min_width=70, value='Solvent')
                [6] StaticText(max_width=70, min_width=70, value='Experiments')
            [1] Row
                [0] StaticText(max_width=80, min_width=80, value='1')
                [1] StaticText(max_width=70, min_width=70, value='Eric')
                [2] StaticText(max_width=70, min_width=70, value='Hughes')
                [3] StaticText(max_width=180, min_width=180, value='eric.hughes@durham.ac.uk'...)
                [4] StaticText(max_width=70, min_width=70, value='qqq')
                [5] StaticText(max_width=70, min_width=70, value='CDCl3')
                [6] StaticText(max_width=70, min_width=70, value=['H1'])
                [7] Button(max_width=60, name='Delete 1')
            [2] Row
                [0] StaticText(max_width=80, min_width=80, value='2')
                [1] StaticText(max_width=70, min_width=70, value='Eric')
                [2] StaticText(max_width=70, min_width=70, value='Hughes')
                [3] StaticText(max_width=180, min_width=180, value='eric.hughes@durham.ac.uk'...)
                [4] StaticText(max_width=70, min_width=70, value='qqq')
                [5] StaticText(max_width=70, min_width=70, value='CDCl3')
                [6] StaticText(max_width=70, min_width=70, value=['H1', 'P31'])
                [7] Button(max_width=60, name='Delete 2')
        [3] FileDownload(callback=<function create_bruker_au..., filename='test.txt', label='Submit')